In [3]:
import pandas as pd


In [4]:
team_stat_path='/Users/tayouthmalla/Documents/Work/basketball-analytical-tool/data-collection/data/raw/team_stats_barttorvik_20250716_211229.csv'
team_ranking_path='/Users/tayouthmalla/Documents/Work/basketball-analytical-tool/data-collection/data/raw/team_rankings_barttorvik_20250716_211229.csv'

In [3]:
# Load your dataset
df = pd.read_csv(team_stat_path)  # replace with your actual file path

# Step 1: Remove rows where 'Rk' column contains 'Rk' (repeated header)
df = df[df['Rk'] != 'Rk']

# Step 2: Remove rows where 'Rk' is NaN and second column (e.g., 'Team') is 'Adj. Eff.'
df = df[~((df['Rk'].isna()) & (df.iloc[:, 1] == 'Adj. Eff.'))]

# Optional: Reset index if needed
df.reset_index(drop=True, inplace=True)

# Save the cleaned dataset
df.to_csv("cleaned_team_stat_dataset.csv", index=False)


In [5]:
# Load your dataset
df2 = pd.read_csv(team_ranking_path)  # replace with your actual file path

# Step 1: Remove rows where 'Rk' column contains 'Rk' (repeated header)
df2 = df2[df2['Rk'] != 'Rk']

# Optional: Reset index if needed
df2.reset_index(drop=True, inplace=True)

print("Successfully removed")


Successfully removed


In [5]:
df2.head()

,Rk,Team,Conf,G,Rec,AdjOE,AdjDE,Barthag,Eff. FG% - EFG%,Eff. FG% - EFGD%,...,FT Rate - FTRD,2-Pt % - 2P%,2-Pt % - 2P%D,3-Pt % - 3P%,3-Pt % - 3P%D,3P Rate - 3PR,3P Rate - 3PRD,Adj T.,WAB,Year
0,1,"Gonzaga 1 seed, Elite Eight",WCC,37,33-4,124.4,91.0,.9732,59.0,44.2,...,25.9,61.4,43.4,36.3,30.4,35.3,34.9,70.3,+7.2,2019
1,2,"Virginia 1 seed, CHAMPS",ACC,38,35-3,123.8,91.1,.9714,55.2,44.7,...,26.3,52.5,45.7,39.5,28.9,39.5,42.5,59.2,+11.1,2019
2,3,"Texas Tech 3 seed, Finals",B12,38,31-7,115.9,86.4,.9670,53.5,43.0,...,36.6,52.8,41.9,36.5,29.7,36.0,40.3,65.9,+7.0,2019
3,4,"Michigan 2 seed, Sweet Sixteen",B10,37,30-7,115.3,86.8,.9634,51.6,44.1,...,24.1,51.8,44.3,34.2,29.1,39.9,30.1,64.4,+9.2,2019
4,5,"Duke 1 seed, Elite Eight",ACC,38,32-6,119.6,90.4,.9618,53.6,45.0,...,24.0,58.0,45.0,30.8,29.9,37.3,35.0,72.2,+11.2,2019


In [12]:
import pandas as pd
import re

def extract_team_seed_result(df, source_col):
    def parse_row(text):
        row = {}
        text = str(text).strip()
        if not text:
            return row
        match = re.search(r'\b(\d+)\b', text)
        if match:
            seed = int(match.group(1))
            parts = text.split(match.group(0), 1)
            team = parts[0].strip()
            after_seed = parts[1].strip()
            after_seed = re.sub(r'^seed[, ]*', '', after_seed, flags=re.IGNORECASE).strip(' ,')
            row['Team'] = team
            row['Seed'] = seed
            if after_seed:
                row['Result'] = after_seed
        else:
            if ',' in text:
                team, result = map(str.strip, text.split(',', 1))
                row['Team'] = team
                row['Result'] = result
            else:
                row['Team'] = text
        return row

    parsed_df = pd.json_normalize(df[source_col].apply(parse_row))
    
    # Drop conflicting columns before joining
    df_clean = df.drop(columns=[col for col in parsed_df.columns if col in df.columns], errors='ignore')
    
    return df_clean.join(parsed_df)


In [13]:
df2 = extract_team_seed_result(df2, 'Team')

In [14]:
df2.head()

,Rk,Conf,G,Rec,AdjOE,AdjDE,Barthag,Eff. FG% - EFG%,Eff. FG% - EFGD%,Turnover% - TOR,...,3-Pt % - 3P%,3-Pt % - 3P%D,3P Rate - 3PR,3P Rate - 3PRD,Adj T.,WAB,Year,Team,Seed,Result
0,1,WCC,37,33-4,124.4,91.0,.9732,59.0,44.2,14.9,...,36.3,30.4,35.3,34.9,70.3,+7.2,2019,Gonzaga,1.0,Elite Eight
1,2,ACC,38,35-3,123.8,91.1,.9714,55.2,44.7,14.7,...,39.5,28.9,39.5,42.5,59.2,+11.1,2019,Virginia,1.0,CHAMPS
2,3,B12,38,31-7,115.9,86.4,.9670,53.5,43.0,17.7,...,36.5,29.7,36.0,40.3,65.9,+7.0,2019,Texas Tech,3.0,Finals
3,4,B10,37,30-7,115.3,86.8,.9634,51.6,44.1,13.9,...,34.2,29.1,39.9,30.1,64.4,+9.2,2019,Michigan,2.0,Sweet Sixteen
4,5,ACC,38,32-6,119.6,90.4,.9618,53.6,45.0,17.5,...,30.8,29.9,37.3,35.0,72.2,+11.2,2019,Duke,1.0,Elite Eight


In [15]:
from datetime import datetime

def fix_misparsed_date(val):
    try:
        if '-00' in val:
            try:
                dt = datetime.strptime(val, '%b-%y')
            except:
                dt = datetime.strptime(val, '%B-%y')
            return f"'{dt.month}-0'"
        
        dt = datetime.strptime(val + '-2000', '%d-%b-%Y')
        return f"'{dt.month}-{dt.day}'"
    
    except:
        return f"'{val}'"

In [16]:
# Add a leading single quote to force Excel to treat it as text
#df2['Rec'] = "'" + df2['Rec']

df2['Rec'] = df2['Rec'].apply(fix_misparsed_date)

# Save the cleaned dataset
df2.to_csv("cleaned_team_ranking_dataset.csv", index=False)

In [10]:
player_stat_path="/Users/tayouthmalla/Documents/Work/basketball-analytical-tool/data-collection/data/raw/players_barttorvik_20250716_211229.csv"

In [11]:
df3= pd.read_csv(player_stat_path, dtype=str)

df3.head()

,Rk,Class Year,Height,Player,Team,Conf,Min%,PRPG!,BPM,ORtg,...,TO,Blk,Stl,FTR,2P,2-point field goal percentage,3P/100,3P,3-point field goal percentage,Year
0,1,Fr,7-Jun,Zion Williamson,Duke,ACC,64.9,6.8,18.7,129.2,...,15.2,5.8,3.9,46.7,272-364,0.747,3.9,24-71,0.338,2019
1,2,Jr,1-Jun,Cassius Winston,Michigan St.,B10,83.3,6.6,9.6,120,...,17.5,0.3,1.8,36.7,159-317,0.502,9.5,84-211,0.398,2019
2,3,Sr,9-May,Chris Clemons,Campbell,BSth,93.9,6.5,8,115.6,...,10.8,1,2.5,42.5,149-266,0.56,19.4,128-370,0.346,2019
3,4,So,3-Jun,Ja Morant,Murray St.,OVC,93.5,6.4,10,115.6,...,22,2.2,2.4,51,194-356,0.545,7.3,56-152,0.368,2019
4,5,Jr,7-Jun,Grant Williams,Tennessee,SEC,78.6,6.1,11.8,124.5,...,15.2,5.3,2,63.7,215-361,0.596,2.2,14-44,0.318,2019


In [12]:
#from datetime import datetime

#def fix_misparsed_date(val):
#    try:
        # Add a dummy year to make parsing safe
#        dt = datetime.strptime(val + '-2000', '%d-%b-%Y')
#        return f"{dt.month}-{dt.day}"
#    except:
#        return val


df3['Height'] = df3['Height'].apply(fix_misparsed_date)
df3['3P'] = df3['3P'].apply(fix_misparsed_date)


In [13]:
df3.head()

,Rk,Class Year,Height,Player,Team,Conf,Min%,PRPG!,BPM,ORtg,...,TO,Blk,Stl,FTR,2P,2-point field goal percentage,3P/100,3P,3-point field goal percentage,Year
0,1,Fr,'6-7',Zion Williamson,Duke,ACC,64.9,6.8,18.7,129.2,...,15.2,5.8,3.9,46.7,272-364,0.747,3.9,'24-71',0.338,2019
1,2,Jr,'6-1',Cassius Winston,Michigan St.,B10,83.3,6.6,9.6,120,...,17.5,0.3,1.8,36.7,159-317,0.502,9.5,'84-211',0.398,2019
2,3,Sr,'5-9',Chris Clemons,Campbell,BSth,93.9,6.5,8,115.6,...,10.8,1,2.5,42.5,149-266,0.56,19.4,'128-370',0.346,2019
3,4,So,'6-3',Ja Morant,Murray St.,OVC,93.5,6.4,10,115.6,...,22,2.2,2.4,51,194-356,0.545,7.3,'56-152',0.368,2019
4,5,Jr,'6-7',Grant Williams,Tennessee,SEC,78.6,6.1,11.8,124.5,...,15.2,5.3,2,63.7,215-361,0.596,2.2,'14-44',0.318,2019


In [14]:
# Save the cleaned dataset
df3.to_csv("cleaned_player_stat_dataset.csv", index=False)

In [15]:
print("successful!!!")

successful!!!
